# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [1]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [3]:
!cat talks.tsv

'cat' is not recognized as an internal or external command,
operable program or batch file.


## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [2]:
talks = pd.read_csv("talks.tsv", sep="\t", header=0)
talks

,title,type,url_slug,venue,date,location,talk_url,description
0,Reducing food waste from farm to fork to benef...,Invited talk,talk-ncsu2022,"North Carolina State University, Plant & Micro...",2022-04-05,"Raleigh, North Carolina",NaN,NaN
1,Food waste: wicked problem or win-win?,Invited talk,talk-gwu2021,"George Washington University, guest lecture in...",2021-11-09,"Washington, DC",NaN,NaN
2,Reducing food waste to benefit environment and...,Invited talk,talk-duke2020,"Duke University, University Program in Ecology...",2020-10-08,"Durham, North Carolina",NaN,NaN
3,Which supply chain stages should we target to ...,Conference presentation,talk-esaussee2019,Ecological Society of America/U.S. Society for...,2019-08-14,"Louisville, Kentucky",NaN,NaN
4,Measuring geodiversity to explain biodiversity...,Conference presentation,talk-iale2018,International Association of Landscape Ecology,2018-04-09,"Chicago, Illinois",NaN,NaN
5,Intraspecific variation reflects drivers of ro...,Conference presentation,talk-esa2017,Ecological Society of America,2017-08-08,"Portland, Oregon",NaN,NaN
6,A globally replicated experiment shows that lo...,Conference presentation,talk-esa2015,Ecological Society of America,2015-08-11,"Baltimore, Maryland",NaN,NaN
7,Food waste impacts on biodiversity,Invited talk,talk-cec2018,Commission on Environmental Cooperation,2018-10-24,"Arlington, Virginia",NaN,at Food Loss and Waste Measurement Experts Gro...
8,Challenges in the functional trait approach to...,Invited talk,talk-idiv2017,German Centre for Integrative Biodiversity Res...,2017-11-15,"Leipzig, Germany",NaN,NaN
9,Intraspecific variation and community assembly,Invited talk,talk-neon2017,"National Ecological Observatory Network, weekl...",2017-01-05,"Boulder, Colorado",NaN,Co-delivered with John Grady


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [3]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

In [4]:
# Fix the issues with publication date (excel)
str(talks['date'])

'0     2022-04-05\n1     2021-11-09\n2     2020-10-08\n3     2019-08-14\n4     2018-04-09\n5     2017-08-08\n6     2015-08-11\n7     2018-10-24\n8     2017-11-15\n9     2017-01-05\n10    2016-09-20\n11    2015-03-01\n12    2014-06-01\n13    2014-01-15\nName: date, dtype: object'

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [5]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    
    if len(str(item.talk_url)) > 3:
        md += "\n[More information here](" + item.talk_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [8]:
!ls ../_talks

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [8]:
!cat ../_talks/2019-08-14-talk-esaussee2019.md

---
title: "Which supply chain stages should we target to reduce food loss and waste and benefit the environment?"
collection: talks
type: "Conference presentation"
permalink: /talks/2019-08-14-talk-esaussee2019
venue: "Ecological Society of America/U.S. Society for Ecological Economics joint meeting"
date: 2019-08-14
location: "Louisville, Kentucky"
---
